In [1]:
import numpy as np
import pandas as pd

In [2]:
# Load data from CSV
doctor_df = pd.read_csv('../Datasets/doctors_data_multireward.csv')

# Specify the doctor speciality for which you want recommendations
speciality_to_select = 'Cardiologist'

doctors_for_speciality = doctor_df[doctor_df['Doctor Speciality'] == speciality_to_select].reset_index(drop=True)

# print(doctors_for_speciality)
doctors_for_speciality.head(5)

,Doctor Speciality,Doctor Name,Ratings,Experience,Distance from Patient,Availability,Cost of Services
0,Cardiologist,Dr. Dennis Ray,3.40,10.44,17.60,0,12.31
1,Cardiologist,Dr. Joel Foster,3.31,1.64,12.54,1,21.59
2,Cardiologist,Dr. John Garza,3.45,2.14,12.81,0,10.91
3,Cardiologist,Dr. Ryan Tate,1.46,2.30,11.98,1,3.69
4,Cardiologist,Dr. Beverly Hampton,3.83,15.15,12.01,1,45.00


In [3]:
patients_df = pd.read_csv('../Datasets/patients_representation.csv')
patients_df.shape

(1000, 770)

<h3>Linear UCB for Contextual bandits</h3>

In [4]:
# pooled approach/ shared approach : A and b are for all arms
class LinUCB:
    def __init__(self, num_arms, num_features, alpha, lambda_reg=0.1):
        self.num_arms = num_arms
        self.num_features = num_features
        self.alpha = alpha
        self.lambda_reg = lambda_reg
        
        self.A = np.identity(num_features)
        self.b = np.zeros((num_features, 1))
        
    def select_arm(self, context):
        max_ucb = float('-inf')
        selected_arm = None
        
        for arm in range(self.num_arms):
            A_inv = np.linalg.inv(self.A + self.lambda_reg * np.identity(self.num_features))
            theta_hat = np.dot(A_inv, self.b)
            ucb = np.dot(theta_hat.T, context) + self.alpha * np.sqrt(np.dot(context.T, np.dot(A_inv, context)))
            
            if ucb > max_ucb:
                max_ucb = ucb
                selected_arm = arm
        
        return selected_arm
    
    def update(self, arm, reward, context):
      context = context.reshape(-1, 1)  # Reshape context to column vector
      self.A += np.dot(context, context.T)
      self.b += reward * context


In [ ]:
num_arms = doctors_for_speciality.shape[0]
# print(num_arms)

num_features = patients_df.shape[1]
context_dim = num_features
num_samples = patients_df.shape[0]
# print(num_samples)
# print(num_features)

contexts = patients_df.copy()
# print(contexts.shape)

rewards = doctors_for_speciality['Ratings'].tolist()
# print(len(rewards))

# Initialize LinUCB algorithm
# alpha = exploration parameter
alpha = 0.1 
linucb = LinUCB(num_arms, num_features, alpha)

# Initialize dictionary to store counts of selections for selected doctors
selected_doctor_counts = {}

# Iterate through samples and update LinUCB
total_reward = 0
for t in range(num_samples):
    context = contexts.iloc[t]
    arm = linucb.select_arm(context)

    # Increment count for the selected doctor
    if arm in selected_doctor_counts:
        selected_doctor_counts[arm] += 1
    else:
        selected_doctor_counts[arm] = 1

    reward = rewards[arm]
    linucb.update(arm, reward, context)
    total_reward += reward

print("Total reward:", total_reward)

# Sort selected doctors based on count of selections in decreasing order
sorted_selected_doctors = sorted(selected_doctor_counts.items(), key=lambda x: x[1], reverse=True)

# Print counts of selections for selected doctors of 'Cardiologist' speciality
print("\nCounts of selections for 'Cardiologist' doctors (sorted by count in decreasing order):")
for doctor_index, count in sorted_selected_doctors:
    doctor_info = doctors_for_speciality.iloc[doctor_index]
    print(f"Doctor Speciality: {doctor_info['Doctor Speciality']}, Doctor Name: {doctor_info['Doctor Name']}, Count: {count}, Ratings: {doctor_info['Ratings']}, Experience: {doctor_info['Experience']}, Distance: {doctor_info['Distance from Patient']}, Availability: {doctor_info['Availability']}, Cost: {doctor_info['Cost of Services']}")